# Kütüphaneler

In [1]:
import tensorflow as tf
import os
import warnings
import time
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.cm as cm
from keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Sequential
from keras.datasets import fashion_mnist
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.keras import layers,activations
from tensorflow.keras.utils import image_dataset_from_directory

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve, KFold, cross_val_score, StratifiedKFold

# Models
import tensorflow
from tensorflow import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras import callbacks
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping

# Image Libraries
from PIL import Image, ImageFilter, ImageStat
import random
random.seed(42)
import PIL
import os
import itertools
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

# Veriyi Yükleme

In [3]:
import tensorflow as tf
import numpy as np
base_dir=r"C:Tensorflow\f1"

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(500,500),
    subset='training',
    batch_size=2
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(500,500),
    subset='validation',
    batch_size=2
)

Found 252 images belonging to 4 classes.
Found 28 images belonging to 4 classes.


# Görselleştirme

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

for _ in range(5):
    img, label = test_datagen.next()
    print(img.shape)   
    plt.imshow(img[0])
    print(label[0])   

    plt.show()

# Wandb ile Eğitim

In [4]:
import wandb
import config
from wandb.keras import WandbCallback

In [5]:
wandb.login()

wandb: Currently logged in as: yasarniyaz (use `wandb login --relogin` to force relogin)


True

In [6]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [1,2]
        },
        'learning_rate': {
            'values': [0.1,0.01]
        },
        'optimizer': {
            'values': ['adam','rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        }
    }
}

In [7]:
sweep_id = wandb.sweep(sweep_config, entity="yasarniyaz", project="sweep")

Create sweep with ID: qn5du4en
Sweep URL: https://wandb.ai/yasarniyaz/sweep/sweeps/qn5du4en


In [8]:
def model_sweep(opt,func):
    
    model= Sequential()

    model.add(layers.Conv2D(filters=4,activation=func,kernel_size=(5,5),input_shape=(500,500,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(filters=8,activation=func,kernel_size=(3,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(filters=16,activation=func,kernel_size=(2,2)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(filters=32,activation=func,kernel_size=(2,2)))

    model.add(layers.Flatten())

    model.add(Dense(50,activation=func))
    model.add(Dense(100,activation=func))
    model.add(Dense(100,activation=func))
    model.add(Dense(50,activation=func))
    model.add(Dense(4,activation="softmax"))
    
    model.compile(optimizer=opt,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [9]:
def train():
    config_defaults = {
        'epochs': 2,
        'batch_size': 128,
        'learning_rate': 0.01,
        'activation': 'relu',
        'optimizer': 'adam',
        'seed': 42
    }

    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    model = model_sweep(config.optimizer,config.activation)
    
    start = time.time()
    model.fit(train_datagen, 
        validation_data=test_datagen,
        batch_size=config.batch_size,
        epochs=config.epochs,
        callbacks=[WandbCallback(data_type="image", validation_data=test_datagen)])
    wandb.log({"training_time":time.time()-start})

In [ ]:
wandb.agent(sweep_id, train)